# NeurIPS 2025 - RDKit + XGBoost Baseline Model

This notebook implements a comprehensive baseline model for the NeurIPS Open Polymer Prediction 2025 competition using:
- RDKit for molecular descriptor generation
- XGBoost for machine learning
- Robust cross-validation strategy
- Feature engineering optimizations

Based on public competition approaches and best practices for molecular property prediction.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb
from scipy.stats import pearsonr
import joblib
from tqdm import tqdm

# RDKit imports
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, Crippen, Lipinski, rdPartialCharges
from rdkit.Chem.rdMolDescriptors import CalcTPSA, CalcLabuteASA, CalcNumRotatableBonds
from rdkit.Chem.Descriptors import ExactMolWt, MolLogP, NumHDonors, NumHAcceptors
from rdkit.Chem.rdmolops import GetFormalCharge

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('default')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Paths
DATA_DIR = Path('../data')
MODELS_DIR = Path('../models')
SUBMISSIONS_DIR = Path('../submissions')

# Create directories if they don't exist
MODELS_DIR.mkdir(exist_ok=True)
SUBMISSIONS_DIR.mkdir(exist_ok=True)

print("Environment setup complete!")
print(f"RDKit version: {Chem.__version__ if hasattr(Chem, '__version__') else 'Available'}")

## Data Loading and Exploration

In [ ]:
# Load data (adjust file names based on actual competition data)
def load_competition_data():
    """Load competition datasets"""
    try:
        # Try common file names
        train_files = ['train.csv', 'training.csv', 'train_data.csv']
        test_files = ['test.csv', 'testing.csv', 'test_data.csv']
        
        train_df = None
        test_df = None
        
        for file in train_files:
            if (DATA_DIR / file).exists():
                train_df = pd.read_csv(DATA_DIR / file)
                print(f"Loaded training data from {file}")
                break
        
        for file in test_files:
            if (DATA_DIR / file).exists():
                test_df = pd.read_csv(DATA_DIR / file)
                print(f"Loaded test data from {file}")
                break
        
        if train_df is None or test_df is None:
            print("Could not find data files. Please ensure data is downloaded.")
            return None, None
        
        print(f"Training data shape: {train_df.shape}")
        print(f"Test data shape: {test_df.shape}")
        
        return train_df, test_df
    
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None

# Load data
train_df, test_df = load_competition_data()

if train_df is not None:
    print("\nTraining data info:")
    print(train_df.info())
    print("\nFirst few rows:")
    display(train_df.head())
    
    print("\nMissing values:")
    print(train_df.isnull().sum())
else:
    print("\n⚠️ No data found. Creating dummy data for demonstration...")
    # Create dummy data for demonstration
    dummy_smiles = [
        'CCO',  # ethanol
        'CC(C)O',  # isopropanol
        'CCCCCCCCCCCCCCCCCC(=O)O',  # stearic acid
        'c1ccccc1',  # benzene
        'CC(C)(C)c1ccc(cc1)O'  # BHT
    ]
    
    train_df = pd.DataFrame({
        'id': range(100),
        'smiles': np.random.choice(dummy_smiles, 100),
        'target': np.random.normal(0, 1, 100)
    })
    
    test_df = pd.DataFrame({
        'id': range(100, 150),
        'smiles': np.random.choice(dummy_smiles, 50)
    })
    
    print("Created dummy data for demonstration.")

## Molecular Feature Engineering with RDKit

In [ ]:
def calculate_molecular_descriptors(smiles_list, descriptor_set='comprehensive'):
    """
    Calculate molecular descriptors from SMILES strings using RDKit.
    
    Parameters:
    -----------
    smiles_list : list
        List of SMILES strings
    descriptor_set : str
        Set of descriptors to calculate ('basic', 'comprehensive', 'all')
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with molecular descriptors
    """
    
    # Define descriptor sets
    basic_descriptors = {
        'MolWt': Descriptors.MolWt,
        'LogP': Descriptors.MolLogP,
        'NumHDonors': Descriptors.NumHDonors,
        'NumHAcceptors': Descriptors.NumHAcceptors,
        'NumRotatableBonds': Descriptors.NumRotatableBonds,
        'NumAromaticRings': Descriptors.NumAromaticRings,
        'TPSA': Descriptors.TPSA,
        'HeavyAtomCount': Descriptors.HeavyAtomCount,
        'NumSaturatedRings': Descriptors.NumSaturatedRings,
        'NumAliphaticRings': Descriptors.NumAliphaticRings
    }
    
    comprehensive_descriptors = {
        **basic_descriptors,
        'BertzCT': Descriptors.BertzCT,
        'Ipc': Descriptors.Ipc,
        'Chi0v': Descriptors.Chi0v,
        'Chi1v': Descriptors.Chi1v,
        'Chi2v': Descriptors.Chi2v,
        'Chi3v': Descriptors.Chi3v,
        'Chi4v': Descriptors.Chi4v,
        'Kappa1': Descriptors.Kappa1,
        'Kappa2': Descriptors.Kappa2,
        'Kappa3': Descriptors.Kappa3,
        'LabuteASA': Descriptors.LabuteASA,
        'BalabanJ': Descriptors.BalabanJ,
        'PEOE_VSA1': Descriptors.PEOE_VSA1,
        'PEOE_VSA2': Descriptors.PEOE_VSA2,
        'PEOE_VSA3': Descriptors.PEOE_VSA3,
        'SMR_VSA1': Descriptors.SMR_VSA1,
        'SMR_VSA2': Descriptors.SMR_VSA2,
        'SMR_VSA3': Descriptors.SMR_VSA3,
        'SlogP_VSA1': Descriptors.SlogP_VSA1,
        'SlogP_VSA2': Descriptors.SlogP_VSA2,
        'SlogP_VSA3': Descriptors.SlogP_VSA3
    }
    
    # Select descriptor set
    if descriptor_set == 'basic':
        descriptors = basic_descriptors
    elif descriptor_set == 'comprehensive':
        descriptors = comprehensive_descriptors
    else:  # 'all'
        # Get all available descriptors
        descriptors = {name: getattr(Descriptors, name) for name in dir(Descriptors) 
                      if not name.startswith('_') and callable(getattr(Descriptors, name))}
    
    results = []
    
    print(f"Calculating {len(descriptors)} descriptors for {len(smiles_list)} molecules...")
    
    for i, smiles in enumerate(tqdm(smiles_list)):
        try:
            mol = Chem.MolFromSmiles(smiles)
            
            if mol is not None:
                # Calculate descriptors
                mol_descriptors = {'SMILES': smiles}
                
                for desc_name, desc_func in descriptors.items():
                    try:
                        value = desc_func(mol)
                        # Handle infinite or NaN values
                        if np.isinf(value) or np.isnan(value):
                            value = 0.0
                        mol_descriptors[desc_name] = value
                    except Exception as e:
                        mol_descriptors[desc_name] = 0.0
                
                # Add custom descriptors
                mol_descriptors['NumRings'] = mol.GetRingInfo().NumRings()
                mol_descriptors['NumAtoms'] = mol.GetNumAtoms()
                mol_descriptors['NumBonds'] = mol.GetNumBonds()
                mol_descriptors['NumHeteroatoms'] = Descriptors.NumHeteroatoms(mol)
                mol_descriptors['FractionCsp3'] = Descriptors.FractionCsp3(mol)
                
                # Lipinski's Rule of Five
                mol_descriptors['Lipinski_Violations'] = (
                    (mol_descriptors['MolWt'] > 500) +
                    (mol_descriptors['LogP'] > 5) +
                    (mol_descriptors['NumHDonors'] > 5) +
                    (mol_descriptors['NumHAcceptors'] > 10)
                )
                
                results.append(mol_descriptors)
            else:
                # Invalid SMILES
                invalid_mol = {'SMILES': smiles}
                for desc_name in descriptors.keys():
                    invalid_mol[desc_name] = 0.0
                results.append(invalid_mol)
                
        except Exception as e:
            print(f"Error processing SMILES {smiles}: {e}")
            continue
    
    feature_df = pd.DataFrame(results)
    print(f"Generated {feature_df.shape[1]-1} molecular descriptors")
    
    return feature_df

# Calculate descriptors for training and test data
# Identify SMILES column (adjust based on actual column name)
smiles_col = None
for col in train_df.columns:
    if 'smiles' in col.lower() or 'molecule' in col.lower():
        smiles_col = col
        break

if smiles_col is None:
    print("Could not identify SMILES column. Please specify manually.")
    # Assume first non-numeric column is SMILES
    smiles_col = train_df.select_dtypes(include=['object']).columns[0]

print(f"Using SMILES column: {smiles_col}")

# Generate features
train_features = calculate_molecular_descriptors(train_df[smiles_col].tolist(), 'comprehensive')
test_features = calculate_molecular_descriptors(test_df[smiles_col].tolist(), 'comprehensive')

print(f"\nTraining features shape: {train_features.shape}")
print(f"Test features shape: {test_features.shape}")

## Feature Engineering and Preprocessing

In [ ]:
def advanced_feature_engineering(features_df):
    """
    Apply advanced feature engineering techniques.
    
    Parameters:
    -----------
    features_df : pd.DataFrame
        DataFrame with molecular descriptors
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with engineered features
    """
    df = features_df.copy()
    
    # Remove SMILES column for feature engineering
    if 'SMILES' in df.columns:
        df = df.drop('SMILES', axis=1)
    
    # Create ratio features
    df['MolWt_per_Atom'] = df['MolWt'] / (df['NumAtoms'] + 1e-6)
    df['TPSA_per_Atom'] = df['TPSA'] / (df['NumAtoms'] + 1e-6)
    df['LogP_per_Atom'] = df['LogP'] / (df['NumAtoms'] + 1e-6)
    df['HeavyAtom_Ratio'] = df['HeavyAtomCount'] / (df['NumAtoms'] + 1e-6)
    df['Aromatic_Ratio'] = df['NumAromaticRings'] / (df['NumRings'] + 1e-6)
    df['Rotatable_Ratio'] = df['NumRotatableBonds'] / (df['NumBonds'] + 1e-6)
    df['HBond_Ratio'] = (df['NumHDonors'] + df['NumHAcceptors']) / (df['NumAtoms'] + 1e-6)
    
    # Create interaction features
    df['LogP_TPSA'] = df['LogP'] * df['TPSA']
    df['MolWt_LogP'] = df['MolWt'] * df['LogP']
    df['Rings_Aromatic'] = df['NumRings'] * df['NumAromaticRings']
    df['HBond_TPSA'] = (df['NumHDonors'] + df['NumHAcceptors']) * df['TPSA']
    
    # Create polynomial features for key descriptors
    key_features = ['MolWt', 'LogP', 'TPSA', 'NumRotatableBonds']
    for feat in key_features:
        if feat in df.columns:
            df[f'{feat}_squared'] = df[feat] ** 2
            df[f'{feat}_sqrt'] = np.sqrt(np.abs(df[feat]))
            df[f'{feat}_log'] = np.log1p(np.abs(df[feat]))
    
    # Create binned features
    df['MolWt_bin'] = pd.cut(df['MolWt'], bins=10, labels=False)
    df['LogP_bin'] = pd.cut(df['LogP'], bins=10, labels=False)
    df['TPSA_bin'] = pd.cut(df['TPSA'], bins=10, labels=False)
    
    # Handle infinite and NaN values
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna(0)
    
    print(f"Feature engineering complete. New shape: {df.shape}")
    return df

# Apply feature engineering
train_features_eng = advanced_feature_engineering(train_features)
test_features_eng = advanced_feature_engineering(test_features)

print(f"\nEngineered training features shape: {train_features_eng.shape}")
print(f"Engineered test features shape: {test_features_eng.shape}")

## Target Variable Analysis and Preprocessing

In [ ]:
# Identify target column
target_col = None
for col in train_df.columns:
    if any(term in col.lower() for term in ['target', 'property', 'value', 'y', 'label']):
        target_col = col
        break

if target_col is None:
    # Assume last numeric column is target
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns
    target_col = numeric_cols[-1]

print(f"Using target column: {target_col}")

# Extract target values
y_train = train_df[target_col]

# Analyze target distribution
print(f"\nTarget statistics:")
print(f"  Count: {len(y_train)}")
print(f"  Mean: {y_train.mean():.4f}")
print(f"  Std: {y_train.std():.4f}")
print(f"  Min: {y_train.min():.4f}")
print(f"  Max: {y_train.max():.4f}")
print(f"  Skewness: {y_train.skew():.4f}")
print(f"  Kurtosis: {y_train.kurtosis():.4f}")

# Plot target distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Histogram
axes[0].hist(y_train, bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Target Distribution')
axes[0].set_xlabel(target_col)
axes[0].set_ylabel('Frequency')

# Box plot
axes[1].boxplot(y_train)
axes[1].set_title('Target Box Plot')
axes[1].set_ylabel(target_col)

# Q-Q plot
from scipy import stats
stats.probplot(y_train, dist="norm", plot=axes[2])
axes[2].set_title('Q-Q Plot (Normal)')

plt.tight_layout()
plt.show()

# Check for outliers
q1 = y_train.quantile(0.25)
q3 = y_train.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = ((y_train < lower_bound) | (y_train > upper_bound)).sum()

print(f"\nOutlier analysis:")
print(f"  IQR: {iqr:.4f}")
print(f"  Lower bound: {lower_bound:.4f}")
print(f"  Upper bound: {upper_bound:.4f}")
print(f"  Number of outliers: {outliers} ({outliers/len(y_train)*100:.2f}%)")

## Feature Selection and Scaling

In [ ]:
# Remove constant features
def remove_constant_features(X_train, X_test):
    """Remove features with zero variance"""
    constant_features = []
    for col in X_train.columns:
        if X_train[col].nunique() <= 1:
            constant_features.append(col)
    
    if constant_features:
        print(f"Removing {len(constant_features)} constant features")
        X_train = X_train.drop(columns=constant_features)
        X_test = X_test.drop(columns=constant_features)
    
    return X_train, X_test

# Remove highly correlated features
def remove_correlated_features(X_train, X_test, threshold=0.95):
    """Remove highly correlated features"""
    corr_matrix = X_train.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find features with correlation greater than threshold
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    
    if to_drop:
        print(f"Removing {len(to_drop)} highly correlated features (threshold: {threshold})")
        X_train = X_train.drop(columns=to_drop)
        X_test = X_test.drop(columns=to_drop)
    
    return X_train, X_test

# Prepare features and target
X_train = train_features_eng.copy()
X_test = test_features_eng.copy()
y = y_train.copy()

print(f"Initial feature shapes: Train {X_train.shape}, Test {X_test.shape}")

# Remove constant features
X_train, X_test = remove_constant_features(X_train, X_test)

# Remove highly correlated features
X_train, X_test = remove_correlated_features(X_train, X_test, threshold=0.95)

print(f"After feature removal: Train {X_train.shape}, Test {X_test.shape}")

# Feature selection using statistical tests
n_features = min(200, X_train.shape[1])  # Select top 200 features or all if fewer
selector = SelectKBest(score_func=f_regression, k=n_features)
X_train_selected = selector.fit_transform(X_train, y)
X_test_selected = selector.transform(X_test)

# Get selected feature names
selected_features = X_train.columns[selector.get_support()]
print(f"\nSelected {len(selected_features)} features using statistical tests")

# Convert back to DataFrame
X_train_final = pd.DataFrame(X_train_selected, columns=selected_features)
X_test_final = pd.DataFrame(X_test_selected, columns=selected_features)

print(f"Final feature shapes: Train {X_train_final.shape}, Test {X_test_final.shape}")

# Display top selected features
feature_scores = selector.scores_[selector.get_support()]
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'score': feature_scores
}).sort_values('score', ascending=False)

print("\nTop 10 selected features:")
print(feature_importance.head(10))

## XGBoost Model Training and Validation

In [ ]:
# XGBoost hyperparameters (optimized for molecular property prediction)
xgb_params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'booster': 'gbtree',
    'max_depth': 6,
    'min_child_weight': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'n_estimators': 1000,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'early_stopping_rounds': 100,
    'verbose': False
}

# Cross-validation setup
cv_folds = 5
kfold = KFold(n_splits=cv_folds, shuffle=True, random_state=RANDOM_STATE)

# Cross-validation training
cv_scores = []
cv_models = []
oof_predictions = np.zeros(len(X_train_final))
test_predictions = np.zeros(len(X_test_final))

print("Starting cross-validation training...")

for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_final)):
    print(f"\nFold {fold + 1}/{cv_folds}")
    
    # Split data
    X_fold_train, X_fold_val = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
    y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
    
    # Train model
    model = xgb.XGBRegressor(**xgb_params)
    model.fit(
        X_fold_train, y_fold_train,
        eval_set=[(X_fold_val, y_fold_val)],
        verbose=False
    )
    
    # Predictions
    val_pred = model.predict(X_fold_val)
    test_pred = model.predict(X_test_final)
    
    # Store results
    oof_predictions[val_idx] = val_pred
    test_predictions += test_pred / cv_folds
    cv_models.append(model)
    
    # Calculate metrics
    fold_rmse = np.sqrt(mean_squared_error(y_fold_val, val_pred))
    fold_mae = mean_absolute_error(y_fold_val, val_pred)
    fold_r2 = r2_score(y_fold_val, val_pred)
    
    cv_scores.append(fold_rmse)
    
    print(f"  RMSE: {fold_rmse:.6f}")
    print(f"  MAE: {fold_mae:.6f}")
    print(f"  R²: {fold_r2:.6f}")

# Overall CV performance
cv_rmse = np.mean(cv_scores)
cv_std = np.std(cv_scores)
oof_rmse = np.sqrt(mean_squared_error(y, oof_predictions))
oof_mae = mean_absolute_error(y, oof_predictions)
oof_r2 = r2_score(y, oof_predictions)

print(f"\n" + "="*50)
print(f"CROSS-VALIDATION RESULTS:")
print(f"  CV RMSE: {cv_rmse:.6f} ± {cv_std:.6f}")
print(f"  OOF RMSE: {oof_rmse:.6f}")
print(f"  OOF MAE: {oof_mae:.6f}")
print(f"  OOF R²: {oof_r2:.6f}")
print(f"="*50)

# Plot actual vs predicted
plt.figure(figsize=(10, 8))
plt.scatter(y, oof_predictions, alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title(f'Out-of-Fold Predictions (R² = {oof_r2:.4f})')
plt.grid(True, alpha=0.3)
plt.show()

# Residual analysis
residuals = y - oof_predictions
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(oof_predictions, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residual Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Feature Importance Analysis

In [ ]:
# Calculate average feature importance across all folds
feature_importance_df = pd.DataFrame({
    'feature': X_train_final.columns,
    'importance': np.mean([model.feature_importances_ for model in cv_models], axis=0)
}).sort_values('importance', ascending=False)

print("Top 20 most important features:")
print(feature_importance_df.head(20))

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(20)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Feature Importance')
plt.title('Top 20 Feature Importance (XGBoost)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Save feature importance
feature_importance_df.to_csv(MODELS_DIR / 'feature_importance.csv', index=False)
print(f"\nFeature importance saved to {MODELS_DIR / 'feature_importance.csv'}")

## Model Saving and Submission Generation

In [ ]:
# Save models and preprocessing objects
model_artifacts = {
    'models': cv_models,
    'feature_selector': selector,
    'selected_features': selected_features.tolist(),
    'cv_scores': cv_scores,
    'oof_predictions': oof_predictions,
    'test_predictions': test_predictions,
    'target_column': target_col,
    'smiles_column': smiles_col
}

joblib.dump(model_artifacts, MODELS_DIR / 'xgb_baseline_models.pkl')
print(f"Models saved to {MODELS_DIR / 'xgb_baseline_models.pkl'}")

# Generate submission file
def create_submission(test_df, predictions, submission_path):
    """Create submission file"""
    # Identify ID column
    id_col = None
    for col in test_df.columns:
        if 'id' in col.lower():
            id_col = col
            break
    
    if id_col is None:
        # Use index as ID
        submission_df = pd.DataFrame({
            'id': range(len(predictions)),
            'prediction': predictions
        })
    else:
        submission_df = pd.DataFrame({
            id_col: test_df[id_col],
            'prediction': predictions
        })
    
    submission_df.to_csv(submission_path, index=False)
    print(f"Submission saved to {submission_path}")
    return submission_df

# Create submission
submission_path = SUBMISSIONS_DIR / 'xgb_baseline_submission.csv'
submission_df = create_submission(test_df, test_predictions, submission_path)

print(f"\nSubmission statistics:")
print(f"  Mean prediction: {test_predictions.mean():.6f}")
print(f"  Std prediction: {test_predictions.std():.6f}")
print(f"  Min prediction: {test_predictions.min():.6f}")
print(f"  Max prediction: {test_predictions.max():.6f}")

# Display first few predictions
print("\nFirst 10 predictions:")
print(submission_df.head(10))

## Model Evaluation and Insights

In [ ]:
# Summary of model performance
print("="*60)
print("MODEL PERFORMANCE SUMMARY")
print("="*60)

print(f"\n📊 DATASET INFORMATION:")
print(f"  • Training samples: {len(X_train_final)}")
print(f"  • Test samples: {len(X_test_final)}")
print(f"  • Features used: {X_train_final.shape[1]}")
print(f"  • Target column: {target_col}")
print(f"  • SMILES column: {smiles_col}")

print(f"\n🎯 MODEL PERFORMANCE:")
print(f"  • Cross-validation RMSE: {cv_rmse:.6f} ± {cv_std:.6f}")
print(f"  • Out-of-fold RMSE: {oof_rmse:.6f}")
print(f"  • Out-of-fold MAE: {oof_mae:.6f}")
print(f"  • Out-of-fold R²: {oof_r2:.6f}")

print(f"\n🔬 FEATURE ENGINEERING:")
print(f"  • Total molecular descriptors: {train_features.shape[1]-1}")
print(f"  • Engineered features: {train_features_eng.shape[1]}")
print(f"  • Selected features: {X_train_final.shape[1]}")
print(f"  • Top feature: {feature_importance_df.iloc[0]['feature']}")

print(f"\n📈 PREDICTIONS:")
print(f"  • Mean prediction: {test_predictions.mean():.6f}")
print(f"  • Prediction std: {test_predictions.std():.6f}")
print(f"  • Prediction range: [{test_predictions.min():.6f}, {test_predictions.max():.6f}]")

print(f"\n💡 KEY INSIGHTS:")
print(f"  • Model shows {'good' if oof_r2 > 0.7 else 'moderate' if oof_r2 > 0.5 else 'poor'} predictive performance")
print(f"  • Feature selection reduced dimensionality by {((train_features_eng.shape[1] - X_train_final.shape[1]) / train_features_eng.shape[1] * 100):.1f}%")
print(f"  • Cross-validation stability: {'good' if cv_std < cv_rmse * 0.1 else 'moderate'}")

print(f"\n📁 SAVED FILES:")
print(f"  • Models: {MODELS_DIR / 'xgb_baseline_models.pkl'}")
print(f"  • Feature importance: {MODELS_DIR / 'feature_importance.csv'}")
print(f"  • Submission: {submission_path}")

print("\n✅ Baseline model training complete!")
print("\n🚀 Next steps:")
print("  1. Hyperparameter tuning with Optuna or similar")
print("  2. Try other algorithms (LightGBM, CatBoost, Neural Networks)")
print("  3. Ensemble different models")
print("  4. Advanced feature engineering (molecular fingerprints, graph features)")
print("  5. External data augmentation")
print("="*60)